In [39]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "ac2280"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read_all({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ModuleSix')

image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Ashli Campbell'))),
    html.Hr(),
    html.Div(className='row',
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, n_clicks_timestamp=0, children='Cats'),
                    html.Button(id='submit-button-two', n_clicks=0, n_clicks_timestamp=0, children='Dogs')
                ]),
    html.Div(className='row',
            style={'display': 'flex'},
                children=[
                    html.Button(id='button-one',n_clicks=0, children= 'Water Rescue'),
                    html.Button(id='button-two',n_clicks=0, children= 'Mountain Rescue'),
                    html.Button(id='button-three',n_clicks=0, children='Disaster Rescue'),
                    html.Button(id='button-four', n_clicks=0, children='Reset')          
        ]

    ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,

    ),
    html.Br(),
    html.Hr(),
#     html.Div(
#         id='map-id',
#         className='col s12 m6',
#         ),
 

#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display':'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(Output('datatable-id',"data"),
             [Input('submit-button-one', 'n_clicks_timestamp'),Input('submit-button-two', 'n_clicks_timestamp'),
                Input('button-one', 'n_clicks'),Input('button-two','n_clicks'), 
                Input('button-three','n_clicks'),Input('button-four','n_clicks')
             ])
def on_click(bt1,bt2,button1,button2,button3,button4):
    buttons_dict = {'submit-button-one': bt1, 'submit-button-two': bt2}
#     button_id = max(buttons_dict, key=buttons_dict.get)
    inverse = [(value, key) for key, value in buttons_dict.items()]
    button_id = max(inverse)[1]
    if 'submit-button-one' == button_id:    
#     if (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.read_all({"animal_type":"Cat"})))
    elif 'submit-button-two' == button_id:   
#     elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog"})))
        
    elif (int(button1) >= 1):
        df = pd.DataFrame.from_records(shelter.read_all({'$and': [ 
            {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
                   {'breed':'Newfoundland'}]}, 
            {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$lte':26, '$gte':156}}]}))
#         button2, button3, button4 = 0 
        
    elif (int(button2)>= 1):
        df = pd.DataFrame.from_records(shelter.read_all({'$and': [ 
            {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
                   {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':26, '$gte':156}}]}))
#         button1, button3, button4 = 0

    elif (int(button3)>=1):
        df = pd.DataFrame.from_records(shelter.read_all({'$and': [ 
            {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
                   {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':20, '$gte':300}}]}))
#         button1, button2, button4 = 0, 0, 0

    elif(int(button4)>=1):
        df = pd.DataFrame.from_records(shelter.read_all({}))
#         button1, button2, button3 = 0
    
    else:
        df = pd.DataFrame.from_records(shelter.read_all({}))
        
    return df.to_dict('records')

# @app.callback(Output('datatable-id','data'),
#             [Input('button-one', 'n_clicks'),Input('button-two','n_clicks'), 
#              Input('button-three','n_clicks'),Input('button-four','n_clicks'),
#             ])
# def update_dashboard(button1,button2,button3,button4):
#     if (int(button1) >= 1):
#         df = pd.Dataframe.from_records(shelter.read_all({'$and': [ 
#             {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
#                    {'breed':'Newfoundland'}]}, 
#             {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
#         button2, button3, button4 = 0

#     elif (int(button2)>= 1):
#         df = pd.Dataframe.from_records(shelter.read_all({'$and': [ 
#             {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
#                    {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]}, 
#             {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
#         button1, button3, button4 = 0

#     elif (int(button3)>=1):
#         df = pd.Dataframe.from_records(shelter.read_all({'$and': [ 
#             {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
#                    {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]}, 
#             {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}))
#         button1, button2, button4 = 0

#     elif(int(button4)>=1):
#         df = pd.Dataframe.from_records(shelter.read_all())
#         button1, button2, button3 = 0
#     df = pd.Dataframe.from_records(shelter.read_all())
#     return df.to_dict('records')
        
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    ##FIX ME ####
#     add code for chart of your choice (e.g. pie chart) #
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, values='age_upon_outcome_in_weeks', names='breed', title='Percentage of breeds available')
        )    
    ]

@app.callback(
   Output('map-id', "children"),
   [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    
    viewDF = pd.DataFrame.from_dict(viewData)
    
    dff = viewDF.loc[0]
    
    lat = viewDF.loc[0,'location_lat']
    long = viewDF.loc[0,'location_long']
    
     # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, long], children=[
                dl.Tooltip(viewDF.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(viewDF.iloc[1,9])
                ])
            ])
        ])
    ]
app